In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/src/.env")

# with the new api
from importnb import imports
with imports("ipynb"):
    from utils import to_timestamp, df_data_sensors, missing_data, correlation_data, df_to_csv, df_from_csv

PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Humidity Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

2023-12-15 15:35:34.197 | DEBUG    | utils:df_from_csv:340 - Load DataFrame: /home/sebaxtian/Workspaces/Tangara/tangara-evaluation/src/data/0_raw/tangaras.csv


Total Tangara Sensors: 22


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_25CE,d29e4r0,D29ESP32DE125CE,3.3789825439453125 -76.54106140136719,3.378983,-76.541061
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_2BDE,d29e6de,D29ESP32DE52BDE,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
3,TANGARA_39D6,d29e6de,D29ESP32DE539D6,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
4,TANGARA_3B7E,d29e6de,D29ESP32DE53B7E,3.3982086181640625 -76.52595520019531,3.398209,-76.525955


In [3]:
# Filter by ID_TANGARA_REFERENCE and IDS_TANGARA_TARGETS

# Tangara Sensor Reference
ID_TANGARA_REFERENCE=os.getenv("ID_TANGARA_REFERENCE", None)
# Tangara Sensors Target
IDS_TANGARA_TARGETS=os.getenv("IDS_TANGARA_TARGETS", None)

print(f"Tangara Sensor Reference: {ID_TANGARA_REFERENCE}")
print(f"Tangara Sensors Target: {IDS_TANGARA_TARGETS}")

ids = IDS_TANGARA_TARGETS.split(',') if IDS_TANGARA_TARGETS else []
ids.append(ID_TANGARA_REFERENCE)

df_tangaras = df_tangaras[df_tangaras['ID'].isin(ids)]

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Tangara Sensor Reference: TANGARA_5636
Tangara Sensors Target: TANGARA_39D6,TANGARA_2BDE,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_422A,TANGARA_5636,TANGARA_D282
Total Tangara Sensors: 8


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
2,TANGARA_2BDE,d29e6de,D29ESP32DE52BDE,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
3,TANGARA_39D6,d29e6de,D29ESP32DE539D6,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
4,TANGARA_3B7E,d29e6de,D29ESP32DE53B7E,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
5,TANGARA_3BEA,d29e6de,D29ESP32DE53BEA,3.3982086181640625 -76.52595520019531,3.398209,-76.525955
6,TANGARA_421A,d29e6de,D29ESP32DE5421A,3.3982086181640625 -76.52595520019531,3.398209,-76.525955


In [4]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

2023-12-15 15:35:34.245 | DEBUG    | utils:to_timestamp:99 - datetime_iso8601: 2023-11-26T00:00:00-05:00, Timestamp: 1700974800000
2023-12-15 15:35:34.246 | DEBUG    | utils:to_timestamp:99 - datetime_iso8601: 2023-12-10T23:59:59-05:00, Timestamp: 1702270799000


Since: 2023-11-26T00:00:00-05:00 -> 1700974800000, Until: 2023-12-10T23:59:59-05:00 -> 1702270799000
Group by Time: 30s


In [5]:
# Humidity Data Frame Sensors
df_hum_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'hum', GROUP_BY_TIME)
df_hum_raw.head()
#df_hum_raw.shape

2023-12-15 15:35:34.255 | DEBUG    | utils:query_measure:192 - sql_query: SELECT last("hum") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE52BDE') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("hum") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE539D6') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("hum") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE53B7E') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("hum") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE53BEA') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("hum") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE5421A') AND time >= 1700974800000ms AND time <= 1702270799000ms GROUP BY time(30s) fill(null); SELECT last("hum") FROM "fixed_stations_01" WHERE ("name" = 'D29ESP32DE5422A') AND time >= 17009

,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_422A,TANGARA_5636,TANGARA_D282
DATETIME,,,,,,,,
2023-11-26 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-26 00:00:30-05:00,55.85,54.69,51.60,49.27,48.37,NaN,NaN,NaN
2023-11-26 00:01:00-05:00,56.08,55.08,51.95,49.83,48.79,NaN,NaN,NaN
2023-11-26 00:01:30-05:00,56.27,55.20,52.10,49.99,48.92,47.22,51.62,52.63
2023-11-26 00:02:00-05:00,56.42,55.43,52.33,50.20,49.15,47.53,51.86,52.84


## Descriptive Statistics

In [6]:
# Describe Data
df_hum_raw.describe()

,TANGARA_2BDE,TANGARA_39D6,TANGARA_3B7E,TANGARA_3BEA,TANGARA_421A,TANGARA_422A,TANGARA_5636,TANGARA_D282
count,28045.000000,25549.000000,39888.000000,39358.000000,39186.000000,38837.000000,38993.000000,39058.000000
mean,50.113499,46.594498,46.447443,45.220562,44.543531,42.787200,45.947322,47.337607
std,8.270983,8.179410,7.904166,7.922073,8.862698,7.288838,7.654836,8.233174
min,33.970000,30.490000,30.620000,29.560000,0.000000,29.080000,31.010000,31.470000
25%,43.130000,39.710000,39.810000,38.900000,38.020000,36.810000,39.690000,40.680000
50%,50.680000,46.650000,46.810000,45.310000,44.425000,42.660000,46.140000,47.430000
75%,56.470000,53.430000,52.280000,50.807500,49.370000,47.920000,51.650000,53.230000
max,78.170000,88.710000,68.340000,71.890000,98.780000,63.250000,69.280000,71.480000


## Missing Data

In [7]:
import warnings
warnings.filterwarnings('ignore')

# Missing Data Threshold
threshold_data = float(os.getenv("THRESHOLD_MISSING_DATA", None))
print(f'Threshold Missing Data: {int(threshold_data * 100)}%')

# For each Tangara sensor
for id_tangara_sensor in df_hum_raw.columns:
    df_hum_sensor = df_hum_raw[[id_tangara_sensor]]
    # To be checked
    to_be_checked, data_percent, missing_data_percent = missing_data(df_hum_sensor, threshold_data)
    if to_be_checked:
        print(f"Tangara Sensor: {id_tangara_sensor}, To be checked: {to_be_checked}, Data: {data_percent}%, Missing Data: {missing_data_percent}%, Threshold: {threshold_data}%")

        # Drop Tangara Sensor from original dataframe
        df_hum_raw = df_hum_raw.drop(id_tangara_sensor, axis=1)

2023-12-15 15:35:55.555 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_2BDE, To be checked: True, Data: 65%, Missing Data: 35%, Threshold: 80%
2023-12-15 15:35:55.558 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_39D6, To be checked: True, Data: 59%, Missing Data: 41%, Threshold: 80%
2023-12-15 15:35:55.561 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_3B7E, To be checked: False, Data: 92%, Missing Data: 8%, Threshold: 80%
2023-12-15 15:35:55.563 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_3BEA, To be checked: False, Data: 91%, Missing Data: 9%, Threshold: 80%
2023-12-15 15:35:55.565 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_421A, To be checked: False, Data: 91%, Missing Data: 9%, Threshold: 80%
2023-12-15 15:35:55.566 | DEBUG    | utils:missing_data:230 - Tangara Sensor: TANGARA_422A, To be checked: False, Data: 90%, Missing Data: 10%, Threshold: 80%
2023-12-15 15:35:55.568 | DEBUG    | utils:missing_

Threshold Missing Data: 80%
Tangara Sensor: TANGARA_2BDE, To be checked: True, Data: 65%, Missing Data: 35%, Threshold: 0.8%
Tangara Sensor: TANGARA_39D6, To be checked: True, Data: 59%, Missing Data: 41%, Threshold: 0.8%


## Data Correlation

In [8]:
# Data Correlation Threshold
threshold_correlation = float(os.getenv("THRESHOLD_CORRELATION_DATA", None))
print(f'Threshold Correlation Data: {int(threshold_correlation * 100)}%')

# Tangara Sensor Reference
ID_TANGARA_REFERENCE=os.getenv("ID_TANGARA_REFERENCE", None)

# Tangara Sensors Target
IDS_TANGARA_TARGETS=os.getenv("IDS_TANGARA_TARGETS", None)
IDS_TANGARA_TARGETS = IDS_TANGARA_TARGETS.split(',') if IDS_TANGARA_TARGETS else ID_TANGARA_REFERENCE

# print(f"Tangara Sensor Reference: {ID_TANGARA_REFERENCE}, Tangara Sensors Target: {IDS_TANGARA_TARGETS}")

# Reference Tangara Sensor
df_tangara_reference = df_hum_raw[[ID_TANGARA_REFERENCE]]

# For each Tangara Sensor Target
for id_tangara in IDS_TANGARA_TARGETS:
    if id_tangara in df_hum_raw.columns:
        # Target Tangara Sensor
        df_tangara_target = df_hum_raw[[id_tangara]]

        # To be checked
        if not math.isnan(df_tangara_reference.std()):
            has_corr, corr = correlation_data(df_tangara_reference, df_tangara_target, threshold_correlation)
            if not has_corr:
                print(f"Tangara Sensor Reference: {ID_TANGARA_REFERENCE}, Tangara Sensors Target: {id_tangara}, There is correlation: {has_corr}, Correlation: {float("{:.2f}".format(corr))}, Threshold: {threshold_correlation}")

                # Drop Tangara Sensor from original dataframe
                df_hum_raw = df_hum_raw.drop(id_tangara, axis=1)

2023-12-15 15:35:55.583 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_3B7E, There is correlation: True, Correlation: 1.0, Threshold: 0.7
2023-12-15 15:35:55.593 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_3BEA, There is correlation: True, Correlation: 0.99, Threshold: 0.7
2023-12-15 15:35:55.622 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_421A, There is correlation: True, Correlation: 0.92, Threshold: 0.7
2023-12-15 15:35:55.635 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_422A, There is correlation: True, Correlation: 1.0, Threshold: 0.7
2023-12-15 15:35:55.644 | DEBUG    | utils:correlation_data:287 - Tangara Sensor Reference: TANGARA_5636, Tangara Sensors Target: TANGARA_5636, There is correlation: True, Correl

Threshold Correlation Data: 70%


In [9]:
# Save Humidity Data Frame Sensors into CSV file
df_to_csv(df_hum_raw, "hum_raw.csv")

2023-12-15 15:35:56.245 | DEBUG    | utils:df_to_csv:311 - Save DataFrame: /home/sebaxtian/Workspaces/Tangara/tangara-evaluation/src/data/0_raw/hum_raw.csv
